# Multimodal search with CLIP

In [1]:
import clip
import pymongo
from superduperdb.misc.superduper import superduper
from superduperdb.models.torch.wrapper import TorchModel
from superduperdb.datalayer.mongodb.query import Collection
from superduperdb.core.documents import Document as D
from superduperdb.encoders.pillow.image import pil_image as i

pymongo.MongoClient().drop_database('documents')
pymongo.MongoClient().drop_database('_filesystem:documents')

db = pymongo.MongoClient().documents
db = superduper(db)

collection = Collection(name='tiny-imagenet')

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


ModuleNotFoundError: No module named 'superduperdb.core.documents'

In [2]:
from superduperdb.core.documents import Document as D
from superduperdb.encoders.pillow.image import pil_image as i
from datasets import load_dataset
import random

dataset = load_dataset("zh-plus/tiny-imagenet")['valid']
dataset = [D({'image': i(r['image'])}) for r in dataset]
dataset = random.sample(dataset, 1000)

ModuleNotFoundError: No module named 'superduperdb.core.documents'

In [3]:
db.execute(collection.insert_many(dataset, encoders=(i,)))

NameError: name 'db' is not defined

In [4]:
x = db.execute(collection.find_one())['image'].x

NameError: name 'db' is not defined

In [5]:
model, preprocess = clip.load("ViT-B/32", device='cpu')

In [6]:
from superduperdb.encoders.torch.tensor import tensor
import torch

t = tensor(torch.float, shape=(512,))

In [7]:
text_model = TorchModel(
    identifier='clip_text',
    object=model,
    preprocess=lambda x: clip.tokenize(x)[0],
    forward_method='encode_text',
    encoder=t
)

In [8]:
text_model.predict('this is a test', one=True)

tensor([ 1.5430e-01,  8.3419e-02, -8.6218e-02, -2.4812e-01, -9.0193e-02,
        -1.7874e-01,  7.9648e-02, -1.4377e+00,  9.5243e-02,  2.9510e-01,
         4.2066e-03,  2.5638e-01, -2.1601e-02,  1.5735e-02,  1.9318e-01,
        -1.6119e-01,  3.6999e-01,  2.2594e-02, -5.7204e-02, -1.9817e-01,
        -7.5301e-03, -3.2254e-01,  2.7348e-01, -3.5396e-01, -1.9037e-01,
         1.0515e-01, -2.4183e-02,  1.0074e-01, -8.8070e-02,  2.6366e-02,
        -2.7968e-01, -2.9460e-01,  2.1931e-02,  1.1822e-01,  8.7239e-02,
         1.1945e-01, -6.0540e-04, -1.0101e-01, -1.0430e-01, -2.0626e-01,
        -1.3790e-01, -3.9173e-02,  7.5546e-03,  1.3866e-01, -1.1169e-01,
         1.1691e-01, -1.8521e-01,  2.9997e-02,  1.0280e-01,  1.1497e-01,
        -1.0389e-01,  5.0075e-02,  3.1277e-01, -1.7357e-01,  4.6908e-02,
        -9.6962e-02,  7.7737e-02,  4.9121e-03, -2.9044e-01, -3.6215e-02,
         3.3161e-01,  4.0403e-02, -3.6717e-02,  3.3114e-02,  1.3841e-01,
         1.1665e-01, -1.8533e-01,  1.4537e-01,  1.7

In [9]:
visual_model = TorchModel(
    identifier='clip_image',
    preprocess=preprocess,
    object=model.visual,
    encoder=t,
)

In [10]:
visual_model.predict(x, one=True)

NameError: name 'x' is not defined

In [11]:
from superduperdb.core.vector_index import VectorIndex
from superduperdb.core.watcher import Watcher

db.add(
    VectorIndex(
        'my-index',
        indexing_watcher=Watcher(
            model=visual_model,
            key='image',
            select=collection.find(),
        ),
        compatible_watcher=Watcher(
            model=text_model,
            key='text',
            active=False,
        )
    )
)

NameError: name 'db' is not defined

In [12]:
out = db.execute(
    collection.like(D({'text': 'mushroom'}), vector_index='my-index', n=3).find({})
)

NameError: name 'db' is not defined

In [13]:
from IPython.display import display
for r in out:
    display(r['image'].x)

NameError: name 'out' is not defined